In [1]:
import numpy as np
import sklearn as skl
import pandas as pd
import sklearn.preprocessing
from sklearn import linear_model
from scipy import special
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

np.random.seed(69) #fixing seed for reproducability

In [54]:
df_train = pd.read_csv("train.csv")
df_splitt = df_train['Sequence'].apply(lambda x: pd.Series(list(x))) #splitts the 4 mutators into individual columns.

#df_letters = pd.concat([df_train['Sequence'].apply(lambda x: pd.Series(list(x))), df_train['Active']],axis=1)

In [55]:
df_test = pd.read_csv("test.csv")
df_test_s = df_test['Sequence'].apply(lambda x: pd.Series(list(x)))

In [56]:
print(df_test_s.shape)

(48000, 4)


In [51]:
# define universe of possible input values
mutators = 'ACDEFGHIKLMNPQRSTUVWY' #abcdefghijklmnopqrstuvwxyz
#bjoqxz <-- these letters are missing

# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(mutators))
int_to_char = dict((i, c) for i, c in enumerate(mutators))

# integer encode input data
#integer_encoded = [char_to_int[item] for item in df_letters[0]]
#print(integer_encoded)

In [53]:
df = np.zeros((112000, 21 * 4))
#Possibly the worst fucking way we could have done this. But fuckem.
for i in range(0,4):
    for j in range(0, 112000):
        temp = char_to_int[df_splitt.loc[j,i]]
        df[j, i*19 + temp] = 1

In [57]:
#Do the same with the test dataframe
df_cock = np.zeros((48000, 21 * 4))

for i in range(0,4):
    for j in range(0, 48000):
        temp = char_to_int[df_test_s.loc[j,i]]
        df_cock[j, i*19 + temp] = 1

In [59]:
clf = skl.linear_model.RidgeCV(alphas=[0.01, 0.01, 0.1, 1, 10, 100, 1000, 2000,5000], cv=15)

labels = df_train['Active'].to_numpy()

#predict cocks:
clf.fit(df,labels)
print('Done with fitting')
predict_train = clf.predict(df)
print('Done with prediction and error is: ', np.sqrt(np.mean((predict_train-labels)**2)))
predict_cancer = clf.predict(df_cock)
predict_cancer = np.where(predict_cancer > 0.5, 1, 0)

0.17186829726659222


In [62]:
clf = skl.svm.LinearSVC(dual=False, class_weight='balanced')

labels = df_train['Active'].to_numpy()

#predict cocks:
clf.fit(df,labels)
print('Done with fitting')
predict_train = clf.predict(df)
print('Done with prediction and error is: ', np.sqrt(np.mean((predict_train-labels)**2)))
predict_cancer = clf.predict(df_cock)
predict_cancer = np.where(predict_cancer > 0.5, 1, 0)

Done with fitting
Done with prediction and error is:  0.31425933330838174


In [67]:
np.savetxt('submission.csv', predict_cancer, delimiter=',', fmt='%i')